In [16]:
from astropy.table import Table
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from minisom import MiniSom   

In [17]:
df_L3_info = Table.read('../dataset/L3_COSMOS2020_Richard_RefCat_2023DEC4_info.fits')
df_L3_info = df_L3_info.to_pandas()

In [18]:
L3_phot_data = np.loadtxt('../dataset/L3_COSMOS2020_Richard_RefCat_2023DEC4_averaged_phot102.txt')
phot_data = L3_phot_data[:,3::2]
phot_err_data = L3_phot_data[:,4::2]

In [2]:
som = MiniSom(6, 6, 4, sigma=0.3, learning_rate=0.5) # initialization of 6x6 SOM
som.train(data, 100) # trains the SOM with 100 iterations

In [3]:
som.winner(data[0])

(np.int64(5), np.int64(4))

In [ ]:
help(MiniSom)

In [24]:
phot_data

array([[  -8.89,   12.76,   26.  , ...,  142.23,  -18.24,  101.69],
       [  18.68,   21.02,   35.54, ...,   28.69,  -41.77,   18.73],
       [  37.59,   67.26,   79.94, ...,   45.31,   29.3 ,   94.04],
       ...,
       [  27.96,   -0.39,   15.97, ...,   28.92,   91.97,   65.44],
       [   3.72,    9.53,   32.91, ...,   12.8 ,   -8.06, -101.04],
       [   9.03,    4.86,   22.86, ...,   -9.34,   97.59,    3.55]])